In [ ]:
######
!pip install mediapipe
######

In [ ]:
from zipfile import ZipFile

# Example: your file is named 'archive(2).zip'
file_name = "/content/My.zip"

# Extract all contents to the current directory
with ZipFile(file_name, 'r') as zip_ref:
    zip_ref.extractall('bdslmy')

print("✅ Unzipping complete! Files are inside the 'bdsl47' folder.")

✅ Unzipping complete! Files are inside the 'bdsl47' folder.


In [ ]:
import cv2
import mediapipe as mp
import os
import csv
import glob

# --- CONFIGURATION ---
# 1. Base directory where your word folders are located
#    Example: /content/bdsl47/011/Beautiful
base_dataset_folder = "/content/bdslmy/"

#W
target_words = ["My"]
# ---------------------

#MediaPipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=1,  #63 points
    min_detection_confidence=0.5
)


headers = ["Single Image Frame"]
for i in range(21):
    headers.extend([f"x{i:02d}", f"y{i:02d}", f"z{i:02d}"])
headers.append("Label")

#Loop
print(f"Processing {len(target_words)} words...")

for word in target_words:

    input_folder = os.path.join(base_dataset_folder, word)
    output_csv = f"Word_{word}_63points.csv"

    if not os.path.exists(input_folder):
        print(f"⚠ Skipping '{word}': Folder not found at {input_folder}")
        continue

    print(f"▶ Creating {output_csv}...")

    # 2. Create/Open the CSV for this word
    with open(output_csv, mode='w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(headers)

        #images
        image_files = []
        for ext in ['*.png', '*.jpg', '*.jpeg']:
            image_files.extend(glob.glob(os.path.join(input_folder, ext)))

        if len(image_files) == 0:
            print(f"   No images found for '{word}'.")
            continue

        #images
        processed_count = 0
        for file_path in image_files:
            try:
                image = cv2.imread(file_path)
                if image is None: continue

                #RGB
                image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                results = hands.process(image_rgb)

                #hand detected
                if results.multi_hand_landmarks:
                    hand_landmarks = results.multi_hand_landmarks[0] # Get first hand


                    row_data = [os.path.basename(file_path)]

                    for lm in hand_landmarks.landmark:
                        row_data.extend([lm.x, lm.y, lm.z])

                    row_data.append(word)

                    writer.writerow(row_data)
                    processed_count += 1

            except Exception as e:
                print(f"   Error on {os.path.basename(file_path)}: {e}")

        print(f"   ✅ Finished '{word}': Saved {processed_count} rows.")

print("-" * 30)
print("All words processed!")

Processing 1 words...
▶ Creating Word_My_63points.csv...
